In [1]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd


In [3]:
season = ["2010-2011","2011-2012", "2012-2013", "2013-2014", "2014-2015", "2015-2016", "2016-2017", "2017-2018", "2018-2019", "2019-2020", "2020-2021", "2021-2022", "2022-2023", "2023-2024"]
headers = {'Accept': '/', 'Connection': 'keep-alive', 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0) AppleWebKit/537.36 (KHTML,like Gecko) Chrome/70.0.3538.110 Safari/537.36', 'Accept-Language':'en-US;q=0.5,en;q=0.3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1'}

In [4]:
url_ranking = "https://www.eurosport.com/football/premier-league/2012-2013/standings.shtml"
url_value = 'https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/plus/?saison_id=2023'


In [5]:
def get_table(headers, url, table_class, season):
    p = requests.get(url, headers = headers) 
    print(p.status_code)
    page = p.content
    soup = BeautifulSoup(page,"html.parser")
    rows = soup.find('table', {'class': table_class}).find('tbody').find_all('tr')
    data = []
    for row in rows:
        td = [cell.get_text(strip=True) for cell in row]
        td = [val for val in td if val]
        data.append(td + [season])
    df = pd.DataFrame(data)
    return df

Original code. Did not want to remove without consensus. 
```python
def get_table(season, headers):
    p = requests.get(f"https://www.eurosport.com/football/premier-league/{season}/standings.shtml", headers = headers) 
    print(p.status_code)
    page = p.content
    soup = BeautifulSoup(page,"html.parser")
    rows = soup.find_all("tr",{"data-testid":"table-row-data"})
    data = []
    for row in rows:
        team_rank = row.find("td",{"data-testid":"table-cell-value"}).text.strip()
        team_name = row.find("td",{"data-testid":"table-cell-team"}).text.strip()
        team_data = row.find_all("td",{"data-testid":"table-cell-value"})
        td = [cell.get_text(strip = True) for cell in team_data]
        data.append([team_rank]+[team_name[:-3]]+td+[season])
    df = pd.DataFrame(data)
    return df
    

get_table("2017-2018")
´´´

In [17]:
table_class_ranking = 'w-full table-auto overflow-hidden border-b border-br-2-80'
full_table = get_table(headers, f"https://www.eurosport.com/football/premier-league/2010-2011/standings.shtml", table_class_ranking, '2010-2011')
for i in season[1:]:
    print(i)
    df = get_table(headers, f"https://www.eurosport.com/football/premier-league/{i}/standings.shtml", table_class_ranking,i)
    full_table = pd.concat([full_table,df])

full_table

200
2011-2012
200
2012-2013
200
2013-2014
200
2014-2015
200
2015-2016
200
2016-2017
200
2017-2018
200
2018-2019
200
2019-2020
200
2020-2021
200
2021-2022
200
2022-2023
200
2023-2024
200


,0,1,2,3,4,5,6,7,8,9,10
0,1,Manchester UnitedMUN,38,23,11,4,78,37,41,80,2010-2011
1,2,ChelseaCHE,38,21,8,9,69,33,36,71,2010-2011
2,3,Manchester CityMCI,38,21,8,9,60,33,27,71,2010-2011
3,4,ArsenalARS,38,19,11,8,72,43,29,68,2010-2011
4,5,Tottenham HotspurTOT,38,16,14,8,55,46,9,62,2010-2011
...,...,...,...,...,...,...,...,...,...,...,...
15,16,BrentfordBRE,38,10,9,19,56,65,-9,39,2023-2024
16,17,Nottingham ForestNOT,38,9,9,20,49,67,-18,32,2023-2024
17,18,Luton TownLUT,38,6,8,24,52,85,-33,26,2023-2024
18,19,BurnleyBUR,38,5,9,24,41,78,-37,24,2023-2024


In [18]:
# saison_id=2023 means season 23/24
s = 2011
table_class_value = 'items'
market_table = get_table(headers, 'https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/plus/?saison_id=2010', table_class_value, '2010-2011')
for i in season[1:]:
    print(s)
    df = get_table(headers, 'https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/plus/?saison_id={}'.format(s), 'items',i)
    market_table = pd.concat([market_table,df])
    s += 1

market_table

200
2011
200
2012
200
2013
200
2014
200
2015
200
2016
200
2017
200
2018
200
2019
200
2020
200
2021
200
2022
200
2023
200


,0,1,2,3,4,5,6
0,Chelsea FC,33,25.9,23,€12.85m,€424.10m,2010-2011
1,Manchester City,45,24.9,28,€8.98m,€404.18m,2010-2011
2,Manchester United,43,25.7,29,€9.02m,€388.00m,2010-2011
3,Liverpool FC,42,24.9,25,€8.08m,€339.20m,2010-2011
4,Arsenal FC,34,24.8,27,€9.53m,€324.00m,2010-2011
...,...,...,...,...,...,...,...
15,Everton FC,34,26.8,16,€11.52m,€391.75m,2023-2024
16,Fulham FC,35,26.9,31,€10.98m,€384.35m,2023-2024
17,Burnley FC,39,25.3,30,€7.01m,€273.48m,2023-2024
18,Sheffield United,45,24.7,26,€3.39m,€152.45m,2023-2024


From here we need to merge the important info from this table into the other via matching club names and season

In [20]:
#removing FC
def clean_market_club_name(club_name):
    return club_name.replace("FC", "").strip()

#removing MUN, CUH etc..
def clean_full_club_name(club_name):
    return club_name[:-3].strip()

#applying above functions
full_table[1] = full_table[1].apply(clean_full_club_name)
market_table[0] = market_table[0].apply(clean_market_club_name)

#renaiming  columns
full_table.columns = ['Position', 'Club', 'P', 'W', 'D', 'L', 'GF', 'GA', '(+/-)', 'PTS', 'Season']
market_table.columns = ['Squad', 'Club', 'Age', 'Foreigners', 'Market Value', 'Total Market Value', 'Season']

print(full_table)
print(market_table)


   Position               Club   P   W   D   L  GF   GA (+/-) PTS     Season
0         1  Manchester United  38  23  11   4  78   37    41  80  2010-2011
1         2            Chelsea  38  21   8   9  69   33    36  71  2010-2011
2         3    Manchester City  38  21   8   9  60   33    27  71  2010-2011
3         4            Arsenal  38  19  11   8  72   43    29  68  2010-2011
4         5  Tottenham Hotspur  38  16  14   8  55   46     9  62  2010-2011
..      ...                ...  ..  ..  ..  ..  ..  ...   ...  ..        ...
15       16          Brentford  38  10   9  19  56   65    -9  39  2023-2024
16       17  Nottingham Forest  38   9   9  20  49   67   -18  32  2023-2024
17       18         Luton Town  38   6   8  24  52   85   -33  26  2023-2024
18       19            Burnley  38   5   9  24  41   78   -37  24  2023-2024
19       20   Sheffield United  38   3   7  28  35  104   -69  16  2023-2024

[280 rows x 11 columns]
                Squad Club   Age Foreigners Market 

In [21]:
#merging
merged_table = pd.merge(full_table, market_table, on=['Club', 'Season'], how='outer')
merged_table.fillna('N/A', inplace=True)
merged_table
#looks bad :S

,Position,Club,P,W,D,L,GF,GA,(+/-),PTS,Season,Squad,Age,Foreigners,Market Value,Total Market Value
0,N/A,27,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,2013-2014,Manchester City,27.3,21,€16.88m,€455.80m
1,N/A,29,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,2011-2012,Norwich City,26.0,12,€2.03m,€58.75m
2,N/A,29,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,2012-2013,Everton,26.2,20,€4.82m,€139.75m
3,N/A,29,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,2017-2018,Southampton,26.4,16,€10.24m,€297.00m
4,N/A,30,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,2011-2012,Wigan Athletic,26.7,22,€2.48m,€74.25m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,7,Wolverhampton Wanderers,38,15,14,9,51,40,11,59,2019-2020,N/A,N/A,N/A,N/A,N/A
556,13,Wolverhampton Wanderers,38,12,9,17,36,52,-16,45,2020-2021,N/A,N/A,N/A,N/A,N/A
557,10,Wolverhampton Wanderers,38,15,6,17,38,43,-5,51,2021-2022,N/A,N/A,N/A,N/A,N/A
558,13,Wolverhampton Wanderers,38,11,8,19,31,58,-27,41,2022-2023,N/A,N/A,N/A,N/A,N/A
